## MNIST Classification

The MNIST dataset is a dataset of handwritten digits that is commonly used as the 'Hello World' dataset in Deep Learning domain. It contains 60,000 training images and 10,000 testing images, and
`carefree-learn` provided a straightforward API to access it.

MNIST dataset can be used for training various image processing systems. In this article, we will focus on how to build our custom models to solve the classification task on MNIST dataset.

In [1]:
# preparations

import torch
import cflearn

import numpy as np
import torch.nn as nn

# MNIST dataset could be prepared with this one line of code
data = cflearn.cv.MNISTData(batch_size=16, transform="to_tensor")

# for reproduction
np.random.seed(142857)
torch.manual_seed(142857)

As shown above, the MNIST dataset could be easily turned into a `DLDataModule` instance, which is the common data interface used in `carefree-learn`.

> The `transform` argument specifies which transform do we want to use to pre-process the input batch. See [`Transforms`](https://carefree0910.me/carefree-learn-doc/docs/user-guides/computer-vision#transforms) for more details.

### Build Model

For demo purpose, we are going to build a simple convolution-based classifier:

In [2]:
@cflearn.register_module("simple_conv")
class SimpleConvClassifier(nn.Sequential):
    def __init__(self, in_channels: int, num_classes: int):
        super().__init__(
            nn.Conv2d(in_channels, 16, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(1),
            nn.Linear(128, num_classes),
        )

We leveraged the [`register_module`](https://carefree0910.me/carefree-learn-doc/docs/developer-guides/computer-vision-customization#customize-models) API here, which can turn a general `nn.Module` instance to a [`IDLModel`](https://carefree0910.me/carefree-learn-doc/docs/design-principles/#model) in `carefree-learn`. After registered, it can be easily accessed with its name (`"simple_conv"`):

In [3]:
cflearn.api.fit_cv(
    data,
    "simple_conv",
    {"in_channels": 1, "num_classes": 10},
    loss_name="cross_entropy",
    metric_names="acc",
    fixed_epoch=1,                                  # for demo purpose, we only train our model for 1 epoch
    cuda=0 if torch.cuda.is_available() else None,  # use CUDA if possible
)

D:\anaconda\envs\cflearn\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Layer (type)                             Input Shape                             Output Shape    Trainable Param #
------------------------------------------------------------------------------------------------------------------------
_                                                                                                                 
  SimpleConvClassifier               [-1, 1, 28, 28]                                 [-1, 10]               98,666
    Conv2d-0                         [-1, 1, 28, 28]                         [-1, 16, 28, 28]                  160
    ReLU-0                          [-1, 16, 28, 28]                         [-1, 16, 28, 28]                    0
    BatchNorm2d-0                   [-1, 16, 28, 28]                         [-1, 16, 28, 28]                   32
    MaxPool2d-0                     [-1, 16, 28, 28]                         [-1, 16, 14, 14]                    0
    Conv2d-1                        [-1, 16, 14, 14]                      

Our model achieves 98.0400% accuracy on validation set within 1 epoch, not bad!